## Computational Chemistry for Experimentalists
## Chapter 13: Electronic Structure Approximations

The core chemical concept of molecular orbitals is formalized in the Hartree-Fock molecular orbital method. Hartree-Fock theory is actually a mean-field theory, replacing the real electron-electron interactions with each electron interacting with the average distribtion of all the others!  This approximation encodes a lot of insight, but it often fails for real chemistry: chemical bonds are too weak, excited states are often too unstable, and van der Waals interactions don't exist at all! A major part of computational chemistry is the development of practical approximations for electron-electron interactions. This set of examples reviews the most widely adopted approximations. . 

In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
import math
import numpy
import matplotlib.pyplot as plt
from pyscf import gto,scf,dft,cc
from pyscf.tools import cubegen
import py3Dmol

## Example 1: N2 Heats of Formation  

The enthalpy of formation is defined in terms of elements in their standard state. The enthalpy of formation of N2(g) is zero. The enthalpy of formation of CO2 is the enthalpy of CO2, minus the enthalpy of O2 (g), minus that of a carbon atom in solid diamond. 

Because standard states are hard, we compute enthalpies of formation using computed atomization energies, and experimental atomic heats of formation. Here we make an energy level diagram of N2 heat of formation computed using various methods. The heat of formation of N atom is 427.68 kJ/mol , from NIST chemistry webbook

In [ ]:
b='def2tzvp'
HFN=427.68
mn=gto.Mole(atom='N',spin=3,basis=b)
mn.build()
mn2=gto.Mole(atom='N 0.0 0.0 0.0; N 0.0 0.0 1.10',spin=0,basis=b)
mn.build()
mn2.build()
mnf=scf.UHF(mn)
mnf.kernel()
mn2f=scf.RHF(mn2)
mn2f.kernel()
AEHF=2625.5*(2*mnf.e_tot-mn2f.e_tot)
mnd=dft.UKS(mn,xc='b3lyp')
mnd.kernel()
mn2d=dft.RKS(mn2,xc='b3lyp')
mn2d.kernel()
AEDFT=2625.5*(2*mnd.e_tot-mn2d.e_tot)
mnd=dft.UKS(mn,xc='pbe,pbe')
mnd.kernel()
mn2d=dft.RKS(mn2,xc='pbe,pbe')
mn2d.kernel()
AEDFT2=2625.5*(2*mnd.e_tot-mn2d.e_tot)

In [ ]:
mnc=cc.CCSD(mnf)
mnc.kernel()
mn2c=cc.CCSD(mn2f)
mn2c.kernel()
AEC=2625.5*(2*mnc.e_tot-mn2c.e_tot)

In [ ]:
fig = plt.figure(figsize = (5, 5))
plt.ylabel("Total energy relative to standard state (kJ/mol)")

plt.axhline(y=0, color='black',linestyle='-')
plt.axhline(y=2*HFN, color='black',linestyle='-')
ax=plt.gca()
ax.text(.1,20,'Standard state')
ax.text(.1,800,'Isolated atoms')

plt.arrow(0,0,0,2*HFN,color='black',width=0.01,head_width=.1,head_length=100,length_includes_head=True)
ax.text(.1,300,'Atomic DHF')

plt.arrow(1,2*HFN,0,-2*HFN,color='black',width=0.01,head_width=.1,head_length=100,length_includes_head=True)
ax.text(.85,300,'Expt')
plt.arrow(1.2,2*HFN,0,-AEHF,color='blue',width=0.01,head_width=.1,head_length=100,length_includes_head=True)
ax.text(1.1,300,'HF',color='blue')
plt.arrow(1.4,2*HFN,0,-AEDFT,color='green',width=0.01,head_width=.1,head_length=100,length_includes_head=True)
ax.text(1.28,-100,'B3LYP',color='green')
plt.arrow(1.6,2*HFN,0,-AEDFT2,color='red',width=0.01,head_width=.1,head_length=100,length_includes_head=True)
ax.text(1.5,-200,'PBE',color='red')
plt.arrow(1.8,2*HFN,0,-AEC,color='gray',width=0.01,head_width=.1,head_length=100,length_includes_head=True)
ax.text(1.7,-50,'CCSD',color='gray')
plt.show()

## Example 2: Dispersion Interactions 
The dispersion (van der Waals) interaction is an explicitly many-body effect. Mean-field Hartree-Fock theory predicts that there is no dispersion, and that (for example) liquid helium does not exist. Here we show how HF, DFT, and waveunction theory near the basis set limit treat helium dimer neergy. 


In [ ]:
b='aug-cc-pvdz'

# Atom energies 
m=gto.Mole(atom='Ne',basis=b)
m.build()
mf=scf.RHF(m)
mf.kernel()
md=dft.RKS(m,xc='b3lyp')
md.kernel()
mc=cc.CCSD(mf)
mc.kernel()

# Dimer energies 
rs=[1.0,1.05,1.1,1.15,1.2,1.3,1.4,1.5,1.6,1.7,1.8,2.0,2.2,2.3,2.5,2.7,3.0,3.2,3.5,3.7,4.0,4.5,5.0,5.5,6.0]
dehf=[]
dedf=[]
decc=[]
for r in rs:
    geom='Ne 0.0 0.0 0.0; Ne 0.0 0.0 %.2f'%(r)
    m2=gto.Mole(atom=geom,basis=b)
    m2.build()
    mf2=scf.RHF(m2)
    mf2.kernel()
    mc2=cc.CCSD(mf2)
    mc2.kernel()
    md2=dft.RKS(m2,xc='b3lyp')
    md2.kernel()
    dehf.append(627.5095*(mf2.e_tot-2*mf.e_tot))
    dedf.append(627.5095*(md2.e_tot-2*md.e_tot))
    decc.append(627.5095*(mc2.e_tot-2*mc.e_tot))

In [ ]:
plt.plot(rs,dehf,color='blue')
plt.plot(rs,dedf,color='red')
plt.plot(rs,decc,color='gray')
plt.xlabel('He2 bond length (Angstrom)')
plt.ylabel('Bond energy (kcal/mol)')
plt.axhline(y=0, color='black',linestyle='-')
ax=plt.gca()
ax.set_ylim(-.2,.1)
plt.show()